# 合并ksrpc+frpc两阻塞代码于单Notebook

In [ ]:
import multiprocessing
import time

from IPython.display import clear_output
from IPython.display import display, Javascript

from ksrpc.utils.process import run_command, ProcessManager


def callback(process_name, stream_type, line, shared_time, shared_count):
    # 更新共享变量
    with shared_time.get_lock():
        shared_time.value = time.perf_counter()
    with shared_count.get_lock():
        i = shared_count.value + 1
        shared_count.value = i

    if i % 50 == 0:
        clear_output(wait=True)
    print(f"[{stream_type}] {process_name}: {line}")


def main():
    # 创建共享变量 (使用'd'表示双精度浮点数)
    shared_time = multiprocessing.Value('d', time.perf_counter())
    shared_count = multiprocessing.Value('i', 0)

    # 创建新回调函数绑定共享变量
    def wrapped_callback(*args):
        callback(*args, shared_time, shared_count)

    p1 = multiprocessing.Process(target=run_command, name="ksrpc",
                                 args=(["python", "-u", "-m", "ksrpc.run_simple"],
                                       wrapped_callback))
    p2 = multiprocessing.Process(target=run_command, name="frpc",
                                 args=(["./frpc", "-c", "./frpc.toml"],
                                       wrapped_callback))

    try:
        ELAPSE = 60 * 5
        print(f"!!!注意：空闲 {ELAPSE} 秒后，应用将退出!!!")
        with ProcessManager(p1, p2):
            while True:
                current_time = time.perf_counter()
                # 读取共享变量值
                with shared_time.get_lock():
                    last_activity = shared_time.value

                print(f"最后活动时间: {last_activity:.0f}, "
                      f"当前时间: {current_time:.0f}, "
                      f"空闲时间: {current_time - last_activity:-3.0f}/{ELAPSE}")

                if current_time - last_activity > ELAPSE:
                    print("!!!空闲时间到，退出!!!")
                    break
                time.sleep(15)
    except KeyboardInterrupt:
        print("主进程已中断")
    except Exception as e:
        print("主进程错误: ", e)
    finally:
        print("所有服务已关闭")
        display(Javascript('IPython.notebook.kernel.interrupt()'))
        print("中断内核")


if __name__ == '__main__':
    main()